In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import bokeh as bk
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas
from bokeh.layouts import gridplot, row, column
from bokeh.models import Div


def plots(default_style):
    if default_style:
        line_style = style = dict()
    else:
        # Somewhat like pandas-bokeh style, though not exactly 100%
        line_style = dict(
            grid=True,
            muted_line_alpha=0,
            height=360,
        )

        style = dict(
            line_alpha=0.1,
            muted_fill_alpha=0,
            **line_style
        )

    ############################################################################
    # Barplot
    ############################################################################
    data = {
        'fruits':
        ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries'],
        '2015': [2, 1, 4, 3, 2, 4],
        '2016': [5, 3, 3, 2, 4, 6],
        '2017': [3, 2, 4, 4, 5, 3]
    }
    df = pd.DataFrame(data).set_index("fruits")
    p_bar = df.hvplot.bar(
        stacked=False,
        ylabel="Price per Unit [€]",
        title="Fruit prices per Year",
        rot=60,
        **style,
    )

    ############################################################################
    # Lineplot
    ############################################################################
    np.random.seed(42)
    df = pd.DataFrame(
        {
            "Company-01": np.random.randn(1000) + 0.2,
            "Company-02": np.random.randn(1000) + 0.17
        },
        index=pd.date_range('1/1/2000', periods=1000)
    )
    df = df.cumsum()
    df = df + 50
    p_line = df.hvplot.line(
        title="Company-01 vs Company-02",
        xlabel="Date",
        ylabel="Stock price [$]",
        yticks=[0, 100, 200, 300, 400],
        ylim=(0, 400),
        colormap=["red", "blue"],
        **line_style,
    )

    ############################################################################
    # Scatterplot
    ############################################################################
    from sklearn.datasets import load_iris
    iris = load_iris()
    df = pd.DataFrame(iris["data"])
    df.columns = iris["feature_names"]
    df["species"] = iris["target"]
    df["species"] = df["species"].map(dict(zip(range(3), iris["target_names"])))
    p_scatter = df.hvplot.scatter(
        x="petal length (cm)",
        y="sepal width (cm)",
        by="species",
        title="Iris DataSet Visualization",
        **style,
    )

    ############################################################################
    # Histogram
    ############################################################################
    df_hist = pd.DataFrame({
        'a': np.random.randn(1000) + 1,
        'b': np.random.randn(1000),
        'c': np.random.randn(1000) - 1
    },
    columns=['a', 'b', 'c'])
    p_hist = df_hist.hvplot.hist(
        alpha=0.5,
        title="Normal distributions",
        **style,
    )

    ############################################################################
    # Custom legend position requires .opt()
    ############################################################################
    # NOTE: need to call .opt() to put legend inside chart area.
    p_all = (p_bar, p_line, p_scatter, p_hist)
    if not default_style:
        p_all = [p.opts(legend_position='top_right') for p in p_all]

    return p_all


def save(fname, p_all):
    ############################################################################
    # Layout and save
    # NOTE: need to convert hv objects to bokeh objects via render().
    ############################################################################
    children = [hv.render(p) for p in p_all]
    grid = column(
        Div(text="<h1>Page Title</h1>"),
        gridplot(children, ncols=2, sizing_mode='stretch_both', merge_tools=False),
    )
    # Unlike pandas-bokeh, this one can operate on existing grid.
    bk.io.save(grid, filename=fname, resources=bk.resources.INLINE, title='Multiple plots')

In [ ]:
save('hvplot-default.html', plots(default_style=True))
save('hvplot-custom.html', plots(default_style=False))